### ipynb文件导出为PPTX格式

In [5]:
from pathlib import Path
import re
import base64
import nbformat
from pptx import Presentation
from pptx.util import Inches, Pt
from PIL import Image

# =======================
# USER CONFIG (只改这里)
# =======================
INPUT_IPYNB = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/00_pipeline/01_stage1/01_README.ipynb")
OUTPUT_PPTX = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/00_pipeline/01_stage1/01_README.pptx")

# 图片在 slide 中的最大宽度/高度（英寸）
IMG_MAX_W = 9.0
IMG_MAX_H = 4.2

# code cell 同时有代码+图：是否把图放同一页（True）还是每张图单独一页（False）
PUT_CODE_OUTPUT_IMAGES_SAME_SLIDE = True
# =======================

# ---------- Utilities ----------
RE_MD_IMAGE = re.compile(r"!\[[^\]]*\]\(([^)]+)\)")
RE_HTML_IMG = re.compile(r'<img[^>]*src=["\']([^"\']+)["\']', re.IGNORECASE)

def safe_mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def decode_output_image(data, mime: str) -> bytes:
    # nbformat output data sometimes already base64 string, sometimes bytes
    if isinstance(data, bytes):
        return data
    if isinstance(data, str):
        # most commonly base64 string
        return base64.b64decode(data)
    # fallback
    raise TypeError(f"Unknown image data type for {mime}: {type(data)}")

def fit_image_size(img_path: Path, max_w_in: float, max_h_in: float, dpi: int = 150):
    """
    Fit image to box while keeping aspect ratio,
    BUT never upscale beyond its native size (at given dpi).
    """
    with Image.open(img_path) as im:
        w_px, h_px = im.size

    # native size in inches at dpi
    native_w_in = w_px / dpi
    native_h_in = h_px / dpi

    # allowed box (also cap by native to avoid upscaling)
    box_w = min(max_w_in, native_w_in)
    box_h = min(max_h_in, native_h_in)

    # if native is tiny, still allow a little scaling? (optional)
    # box_w = min(max_w_in, max(native_w_in, 4.0))
    # box_h = min(max_h_in, max(native_h_in, 3.0))

    aspect = w_px / h_px
    box_aspect = box_w / box_h

    if aspect >= box_aspect:
        w_in = box_w
        h_in = box_w / aspect
    else:
        h_in = box_h
        w_in = box_h * aspect

    return w_in, h_in


# ---------- Minimal Markdown -> PPT rich text ----------
# 支持：标题(#)、无序/有序列表、粗体/斜体、行内代码、引用块
RE_BOLD = re.compile(r"\*\*(.+?)\*\*")
RE_ITALIC = re.compile(r"(?<!\*)\*(?!\s)(.+?)(?<!\s)\*(?!\*)")
RE_CODE = re.compile(r"`(.+?)`")

def parse_inline_runs(text: str):
    """
    Parse inline markdown into runs: (chunk, style_dict)
    style_dict keys: bold, italic, code
    """
    runs = []
    i = 0
    while i < len(text):
        # find next token among **, *, `
        next_pos = len(text)
        next_kind = None
        next_match = None

        for kind, regex in [("code", RE_CODE), ("bold", RE_BOLD), ("italic", RE_ITALIC)]:
            m = regex.search(text, i)
            if m and m.start() < next_pos:
                next_pos = m.start()
                next_kind = kind
                next_match = m

        if next_match is None:
            runs.append((text[i:], {}))
            break

        if next_match.start() > i:
            runs.append((text[i:next_match.start()], {}))

        content = next_match.group(1)
        if next_kind == "code":
            runs.append((content, {"code": True}))
        elif next_kind == "bold":
            runs.append((content, {"bold": True}))
        elif next_kind == "italic":
            runs.append((content, {"italic": True}))

        i = next_match.end()

    return runs

def add_markdown_textbox(slide, md: str, left, top, width, height):
    """
    Render markdown into a single textbox with basic formatting.
    """
    box = slide.shapes.add_textbox(left, top, width, height)
    tf = box.text_frame
    tf.clear()
    tf.word_wrap = True

    lines = md.splitlines()

    first_para = True
    for raw in lines:
        line = raw.rstrip("\n")

        # blank line -> add empty paragraph to keep spacing
        if line.strip() == "":
            p = tf.add_paragraph() if not first_para else tf.paragraphs[0]
            if first_para:
                tf.paragraphs[0].text = ""
                first_para = False
            else:
                p.text = ""
            p.font.size = Pt(14)
            continue

        # Heading
        m = re.match(r"^(#{1,6})\s+(.*)$", line)
        if m:
            level = len(m.group(1))
            text = m.group(2).strip()
            p = tf.add_paragraph() if not first_para else tf.paragraphs[0]
            if first_para:
                tf.paragraphs[0].text = ""
                first_para = False
            p.level = 0
            p.font.bold = True
            # sizes for headings
            size_map = {1: 30, 2: 26, 3: 22, 4: 18, 5: 16, 6: 14}
            p.font.size = Pt(size_map.get(level, 18))

            # inline styles in headings
            for chunk, st in parse_inline_runs(text):
                run = p.add_run()
                run.text = chunk
                run.font.size = Pt(size_map.get(level, 18))
                run.font.bold = True or st.get("bold", False)
                run.font.italic = st.get("italic", False)
                if st.get("code", False):
                    run.font.name = "Courier New"
            continue

        # Blockquote
        if line.lstrip().startswith(">"):
            text = line.lstrip()[1:].lstrip()
            p = tf.add_paragraph() if not first_para else tf.paragraphs[0]
            if first_para:
                tf.paragraphs[0].text = ""
                first_para = False
            p.level = 0
            p.font.size = Pt(14)
            p.font.italic = True
            p.text = text
            continue

        # List item (unordered/ordered) with indentation
        # Detect leading spaces for nesting (2 spaces per level)
        leading_spaces = len(line) - len(line.lstrip(" "))
        indent_level = leading_spaces // 2

        ul = re.match(r"^\s*[-\*\+]\s+(.*)$", line)
        ol = re.match(r"^\s*\d+\.\s+(.*)$", line)
        if ul or ol:
            text = (ul.group(1) if ul else ol.group(1)).strip()
            p = tf.add_paragraph() if not first_para else tf.paragraphs[0]
            if first_para:
                tf.paragraphs[0].text = ""
                first_para = False
            p.level = min(indent_level, 5)
            p.font.size = Pt(14)

            # Add bullet (pptx uses bullet by default when level>0 sometimes; force bullet)
            p.text = ""  # we will add runs
            p.font.size = Pt(14)

            for chunk, st in parse_inline_runs(text):
                run = p.add_run()
                run.text = chunk
                run.font.size = Pt(14)
                run.font.bold = st.get("bold", False)
                run.font.italic = st.get("italic", False)
                if st.get("code", False):
                    run.font.name = "Courier New"
            continue

        # Normal paragraph
        p = tf.add_paragraph() if not first_para else tf.paragraphs[0]
        if first_para:
            tf.paragraphs[0].text = ""
            first_para = False
        p.level = 0
        p.font.size = Pt(14)
        p.text = ""  # fill runs

        for chunk, st in parse_inline_runs(line.strip()):
            run = p.add_run()
            run.text = chunk
            run.font.size = Pt(14)
            run.font.bold = st.get("bold", False)
            run.font.italic = st.get("italic", False)
            if st.get("code", False):
                run.font.name = "Courier New"

# ---------- Slide builders ----------
prs = Presentation()
BLANK = prs.slide_layouts[6]

assets_dir = OUTPUT_PPTX.with_suffix("")  # folder next to pptx name
safe_mkdir(assets_dir)

def add_title(slide, title: str):
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9.0), Inches(0.6))
    tf = title_box.text_frame
    tf.clear()
    p = tf.paragraphs[0]
    p.text = title
    p.font.size = Pt(20)
    p.font.bold = True

def add_code_box(slide, code: str, top_in: float, height_in: float):
    box = slide.shapes.add_textbox(Inches(0.5), Inches(top_in), Inches(9.0), Inches(height_in))
    tf = box.text_frame
    tf.clear()
    tf.word_wrap = False

    lines = code.splitlines()
    for i, ln in enumerate(lines):
        p = tf.add_paragraph() if i > 0 else tf.paragraphs[0]
        p.text = ln
        p.font.name = "Courier New"
        p.font.size = Pt(12)

def add_picture(slide, img_path: Path, top_in: float):
    w_in, h_in = fit_image_size(img_path, IMG_MAX_W, IMG_MAX_H)
    left_in = (10.0 - w_in) / 2  # slide width ~10in in default template
    slide.shapes.add_picture(str(img_path), Inches(left_in), Inches(top_in), width=Inches(w_in), height=Inches(h_in))
    return top_in + h_in + 0.2

def extract_md_images_from_cell(cell, cell_idx: int, nb_dir: Path):
    """
    Return list of local image paths referenced in markdown, including:
    - ![](relative/or/absolute/path.png)
    - ![](attachment:image.png)  (stored in cell.attachments)
    """
    md = cell.get("source", "") or ""
    paths = []

    # --- 1) Markdown ![]() ---
    for m in RE_MD_IMAGE.finditer(md):
        src = m.group(1).strip().strip('"').strip("'")

        # (a) attachment:xxx
        if src.startswith("attachment:"):
            name = src.split("attachment:", 1)[1]
            attachments = cell.get("attachments", {}) or {}
            if name in attachments:
                att = attachments[name]  # dict: {mime: base64str}
                # pick first image mime
                mime = next((k for k in att.keys() if k.startswith("image/")), None)
                if mime:
                    suffix = mime.split("/")[-1].replace("jpeg", "jpg")
                    out_path = assets_dir / f"cell{cell_idx:03d}_att_{name}"
                    # ensure suffix
                    if not out_path.suffix:
                        out_path = out_path.with_suffix("." + suffix)
                    data_b64 = att[mime]
                    out_path.write_bytes(base64.b64decode(data_b64))
                    paths.append(out_path)
            continue

        # (b) skip urls / data uris
        if src.startswith(("http://", "https://", "data:")):
            continue

        # (c) local file path
        p = (nb_dir / src).resolve() if not Path(src).is_absolute() else Path(src)
        if p.exists():
            paths.append(p)

    # --- 2) HTML <img src=""> ---
    for m in RE_HTML_IMG.finditer(md):
        src = m.group(1).strip()

        if src.startswith("attachment:"):
            name = src.split("attachment:", 1)[1]
            attachments = cell.get("attachments", {}) or {}
            if name in attachments:
                att = attachments[name]
                mime = next((k for k in att.keys() if k.startswith("image/")), None)
                if mime:
                    suffix = mime.split("/")[-1].replace("jpeg", "jpg")
                    out_path = assets_dir / f"cell{cell_idx:03d}_att_{name}"
                    if not out_path.suffix:
                        out_path = out_path.with_suffix("." + suffix)
                    out_path.write_bytes(base64.b64decode(att[mime]))
                    paths.append(out_path)
            continue

        if src.startswith(("http://", "https://", "data:")):
            continue

        p = (nb_dir / src).resolve() if not Path(src).is_absolute() else Path(src)
        if p.exists():
            paths.append(p)

    return paths

def extract_output_images(cell, cell_idx: int):
    img_paths = []
    for out_idx, output in enumerate(cell.get("outputs", []), start=1):
        data = output.get("data", {})
        for mime in ("image/png", "image/jpeg", "image/jpg", "image/webp"):
            if mime in data:
                suffix = mime.split("/")[-1].replace("jpeg", "jpg")
                img_bytes = decode_output_image(data[mime], mime)
                p = assets_dir / f"cell{cell_idx:03d}_out{out_idx:02d}.{suffix}"
                p.write_bytes(img_bytes)
                img_paths.append(p)
                break
    return img_paths

# ---------- Main ----------
nb = nbformat.read(str(INPUT_IPYNB), as_version=4)
nb_dir = INPUT_IPYNB.parent.resolve()

for idx, cell in enumerate(nb.cells, start=1):
    slide = prs.slides.add_slide(BLANK)
    add_title(slide, f"Cell {idx} ({cell.cell_type})")

    if cell.cell_type in ("markdown", "raw"):
        md = cell.source or ""
        # 1) text block
        add_markdown_textbox(
            slide,
            md,
            left=Inches(0.5),
            top=Inches(1.0),
            width=Inches(9.0),
            height=Inches(3.2),
        )
        # 2) images referenced by markdown
        img_paths = extract_md_images_from_cell(cell, idx, nb_dir)
        y = 4.3
        for p in img_paths:
            y = add_picture(slide, p, top_in=y)

    elif cell.cell_type == "code":
        code = cell.source or ""
        # put code box
        add_code_box(slide, code, top_in=Inches(1.0).inches, height_in=3.0)

        img_paths = extract_output_images(cell, idx)
        if PUT_CODE_OUTPUT_IMAGES_SAME_SLIDE:
            y = 4.2
            for p in img_paths:
                y = add_picture(slide, p, top_in=y)
        else:
            # code slide only; each image gets its own slide
            for p in img_paths:
                s2 = prs.slides.add_slide(BLANK)
                add_title(s2, f"Cell {idx} output image")
                add_picture(s2, p, top_in=1.1)

# save
OUTPUT_PPTX.parent.mkdir(parents=True, exist_ok=True)
prs.save(str(OUTPUT_PPTX))
print(f"✅ Saved PPTX: {OUTPUT_PPTX}")
print(f"🖼️ Extracted images saved in: {assets_dir}")


✅ Saved PPTX: /home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/00_pipeline/01_stage1/01_README.pptx
🖼️ Extracted images saved in: /home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/00_pipeline/01_stage1/01_README
